<p style = "font-size : 50px; color : blue ; font-family : 'Comic Sans MS'; text-align : center; background-color : yellow; border-radius: 5px 5px;"><strong>Objective</strong></p>

1. Basic EDA includes most participated player. Most wins by a team (not only finals).
2. Advance EDA Best performing Players though out the IPL all seasons.

<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #F4E0FF; border-radius: 5px 5px;"><strong>Try to find out answers of</strong></p>
1. Is there any correlation between Venue and Winning team
2. Does Toss affect match outcome
3. Does Batting 1st or 2nd impact the outcome?
4. 

In [ ]:
import numpy as np # linear algebra
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Packages for Visualisation 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
# When using the 'inline' backend, your matplotlib graphs will be included in your notebook, next to the code.



In [ ]:
# Importing packages for Predictions
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm

from sklearn.ensemble import GradientBoostingClassifier

from sklearn import tree

In [ ]:
# Defining a function to print side by side
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

# this works 
#df1 = pd.DataFrame(np.arange(12).reshape((3,4)),columns=['A','B','C','D',])
#df2 = pd.DataFrame(np.arange(16).reshape((4,4)),columns=['A','B','C','D',])

#df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('Caption table 1')
#df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('Caption table 2')

#display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

<p style = "font-size : 25px; color : grey ; font-family : 'Comic Sans MS'; text-align : center; background-color : null; border-radius: 5px 5px;"><strong>1. Data Loading & basic analysis </strong></p>

In [ ]:
byb=pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv')
match= pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv')

# Change the name of Delhi Daredevils to Delhi Capital

In [ ]:
# ball by ball
byb

In [ ]:
# Match Data set
match

In [ ]:
print(byb.columns)
print(match.columns)

In [ ]:
byb.info()

In [ ]:
match.info()

Method have a lot of null values lets check what are the unique value does method have.

* So it turns our in 19 matches outcome was decided by D/L method.

The Duckworth–Lewis–Stern method (DLS) is a mathematical formulation designed to calculate the target score for the team batting second in a limited overs cricket match interrupted by weather or other circumstances. The method was devised by two English statisticians, Frank Duckworth and Tony Lewis, and was formerly known as the Duckworth–Lewis method (D/L).[1] It was introduced in 1997, and adopted officially by the ICC in 1999. 

In [ ]:
# Checking match win total null values
match['winner'].isnull().sum()

In [ ]:
# Checking null match values
# I Checked the internet these matches were called off
# So we could simply removes these columns

match[match['winner'].isnull()==True]

In [ ]:
match[match['result']=='tie']
# So far 13 match are tie and we can see eliminator column is having Y  

In [ ]:
match.method.unique()

In [ ]:
# Check the no match each team won
match['winner'].value_counts()

In [ ]:
match['team1'].value_counts()

In [ ]:
match['team2'].value_counts()

1. We need to merge Delhi capitals and Delhi DareDevil into **Delhi Capital** as it has been renamed to Delhi Capital.
2. There is problem with Rising Pune Supergiant as 's' is added at some places.


<p style = "font-size : 25px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #D07FFF; border-radius: 5px 5px;"><strong>2.Exploratory Data Analysis (EDA)</strong></p> 
Exploratory Data Analysis refers to the critical process of performing initial investigations on data so as to discover patterns,to spot anomalies,to test hypothesis and to check assumptions with the help of summary statistics and graphical representations.

In [ ]:
# Most wins in IPL
temp = pd.DataFrame({'Winner': match['winner']})
count_win = temp.value_counts()
# Count_wins is a dictionary; with key being Mumbai Indian and all

# creating label : containing team's name

labels= [I[0] for I in count_win.keys()]


#count_win.bar()
'''plt.bar(count_win)
plt.show()'''

bar, ax= plt.subplots(figsize=(10,10))
ax=plt.pie(x=count_win, autopct="%.1f", labels= labels)
plt.title("Wins")
plt.show()

In [ ]:
# Top 10 Umpire1 & 2 value count

sns.barplot(x=match['umpire1'].value_counts().head(10).values,
            y=match['umpire1'].value_counts().head(10).index,
            data= match
           )
plt.xlabel("No of Match")
plt.ylabel("Name of Umpire")
plt.title('10 Most participated Umpire1')
plt.show()

In [ ]:
# Playing with date
match['Season']= pd.DatetimeIndex(match['date']).year
match.head(2)

In [ ]:
match_per_season = match.groupby(['Season'])['id'].count().reset_index().rename(columns={'id':'matches'})

# {'id':'matches'}: Dictionary to convert id columns to matches
# match_per_season
x= match_per_season.Season
y=match_per_season.matches
mps=match_per_season

plt.bar(match_per_season.Season, match_per_season.matches)
plt.xticks(rotation=90)
plt.grid(True)
plt.xticks(list(range(1,max(x)+1)),[str(i) for i in range(1,max(x)+1)], fontsize=15) # To plot X axis with 1 interval
plt.yticks(np.arange(0,80,step=10), fontsize=15) # Here again we are setting frequency using array
plt.xlim([2007, 2021]) # to define limit of x
plt.ylim([0, 80])
plt.xlabel("Season", fontsize=15)
plt.ylabel("No of Matches", fontsize=15)
plt.title('Total number matches played in each season', fontsize=15, fontweight = 'bold')
plt.show()

# Rendering dataframe as matplotlib table
'''ax2 = fig.add_subplot(122)
font_size=14
bbox=[0, 0, 1, 1]
ax2.axis('off')
mpl_table = ax2.table(cellText = mps.values, rowLabels = mps.index, bbox=bbox, colLabels=mps.columns)
mpl_table.auto_set_font_size(False)
mpl_table.set_fontsize(font_size)
'''

In [ ]:
# Mergring season column with byb using lefet joint
season_data= match[['id','Season']].merge(byb,left_on='id',right_on='id',how='left').drop('id', axis=1)
season_data.head(2)

In [ ]:
# total Run scored in each season. Total Season Run= sr
# Avg run per Season. average run =ars
# msp= Match per Season

sr = season_data.groupby(['Season'])['total_runs'].sum().reset_index()
ars= pd.concat([mps,sr.iloc[:,1]],axis=1)
ars['Runs scored per match']=ars['total_runs']/ars['matches']
ars.set_index('Season',inplace=True)

# Sub plot in sns

f, axes = plt.subplots(1, 2)
f.set_size_inches(17, 5)
axes[0].grid(True)
axes[1].grid(True)
plt.xticks(fontsize=12)
sns.barplot(x=ars.index,y=ars['total_runs'], data= ars, ax=axes[0]);
sns.lineplot(x=ars.index,y=ars['Runs scored per match'], data= ars, ax=axes[1]);

# ; avoids excess text output

<p style = "font-size : 15px; color : #f5f5f0 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00004d; border-radius: 5px 5px;">Venue and 1st inning</p>


In [ ]:
# Venue where atleast 10 matches have been played

sns.set_context("talk")
fig, ax = plt.subplots(figsize=(12, 7))
sns.barplot(x=match['venue'].value_counts().head(22).values,
           y= match['venue'].value_counts().head(22).index,
           data=match)

plt.title('Venue where atleast 10 matches have been played')
plt.xlabel('Venue Count')
plt.ylabel('Venue')
plt.show()

In [ ]:
# Venue where atmax 10 matches have been played

sns.set_context("talk")
fig, ax = plt.subplots(figsize=(12, 7))
sns.barplot(x=match['venue'].value_counts().tail(15).values,
           y= match['venue'].value_counts().tail(15).index,
           data=match)

plt.title('Venue where atmax 10 matches have been played')
plt.xlabel('Venue Count')
plt.ylabel('Venue')
plt.show()

In [ ]:
# Which Stadium is best for winning by wickets i.e team fielding won the match
match.venue[match.result!='runs'].mode()

In [ ]:
# Which stadium is best for winning by run i.e bat First
match.venue[match.result=='runs'].mode()

### Which stadium is best for team when they win the toss
* Principle code
> match.venue[match.toss_winner == 'Kolkata Knight Riders'][match.winner=='Kolkata Knight Riders'].mode()
* Mode : **The mode is the value that appears most frequently in a data set**. A set of data may ***have one mode, more than one mode, or no mode at all***. Other popular measures of central tendency include the mean, or the average of a set, and the median, the middle value in a set.

Refer **List and Dir function** Dir function to check the attributes of an object.

#### Try
a= match.venue[match.toss_winner == 'Kolkata Knight Riders'][match.winner=='Kolkata Knight Riders'].mode()
#print(dir(a))
print(a.values[0])
#print(a.to_string)

In [ ]:
# Which stadium is best for team when they win the toss

# Create a list containing complete team name
t= match['team1'].unique()
for i in t:
    venues= match.venue[match.toss_winner == i][match.winner==i].mode()
    for venue in venues:
        print(venue +' is best for '+ i + ' if they won the toss')
        #print()
    print()


# Diagnosis
'''print(a +' is best for '+ i + ' if they won the toss')
    print()'''
    # print(a.values[0])
  #print(venues.values[0] +' is best for '+ i + ' if they won the toss')
    #print()

In [ ]:
# Which is the best chasing team
match.winner[match.result!='runs'].mode()

In [ ]:
# Which Team won the most no of matches batting 1st
match.winner[match.result=='runs'].mode()

In [ ]:
match['result'].value_counts()

<p style = "font-size : 15px; color : #f5f5f0 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00004d; border-radius: 5px 5px;"></p>
​

<p style = "font-size : 15px; color : #ff3300 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ffff33; border-radius: 7px 5px;"><Strong>Toss ka Funda</Strong></p>

In [ ]:
# Toss Decision
teams= match['toss_winner'].unique()
decision_making=pd.DataFrame([],columns =['Toss Winner','Decision', 'Times'])

for id, element in enumerate(teams):
    temp_bat=match[(match['toss_winner']== element) & (match['toss_decision']== 'bat')]
    temp_field= match[(match['toss_winner']== element)& (match['toss_decision']=='field')]
    
    #append to decision making
    decision_making = decision_making.append({'Toss Winner': element, 'Decision':'bat', 
                                              'Times' :temp_bat['toss_winner'].count()},ignore_index= True)
    
    # For Feild
    decision_making = decision_making.append({'Toss Winner': element, 'Decision':'field', 
                                              'Times' :temp_field['toss_winner'].count()},ignore_index= True)

In [ ]:
decision_making

In [ ]:
# Toss won vs match won
toss= match['toss_winner']==match['winner']
# who won more, 1 who choose batting or the one who choose fielding
decision= match.toss_decision[match.toss_winner==match.winner]

print('Toss fig 1')
f, ax =  plt.subplots(1,2,figsize=(8,3.5))
sns.countplot(toss,ax=ax[0])
sns.countplot(decision ,ax=ax[1])
plt.show()

In [ ]:
# Tosses won by each team &
toss= match['toss_winner'].value_counts()
# sns.set(rc={'figure.figsize':(12,8)}, style='darkgrid')

# No of match won by each team
win=match.winner.value_counts()

print('Toss fig 2')

f, ax =  plt.subplots(1,2,figsize=(20,8))

sns.set(font_scale=1.2)  # to resize x & y ticks just remember its scale not size
sns.barplot(y=toss.index, x= toss, orient = 'h', saturation=1, ax=ax[0])
ax[0].set_title('No. of toss won by each team', fontweight='bold', fontsize=15 )
sns.barplot(y=win.index, x= win, orient = 'h', saturation=1, ax=ax[1])
ax[1].set_title('No. of match won by each team', fontweight='bold', fontsize=15 )
# plt.yticks(fontsize=12)
plt.show()

In [ ]:
print('Toss fig 3')
ax= plt.axes()
sns.set(rc={'figure.figsize':(12,8)}, style='darkgrid')
ax.set_title('Toss decision across season', fontsize=15, fontweight='bold')
sns.countplot(x='Season', hue='toss_decision' ,data = match, saturation=1)
plt.xlabel("Season")
plt.ylabel("Count")
plt.show()

### Conclusion of Toss ka Funda
**fig 1** : there is very slight edge to toss winning team. Plus we also observe that *team who won toss and choose to field have more chance of winning the match.*

**fig 2 :** So does it mean winning the toss is directly proportional to winning the match as both graph seems correlated?
> **No**, see the previous graph we can see the exact relationship between match won and toss won. It certainly gives edge but very minor edge.

**fig 3 :** There is a trend seen in matches between 2016-2019 where team prefer to choose fielding rather than batting. How ever in 2020 this trend seem to fade.

In [ ]:
# player winning the man of the match max no of time
man_of_the_match= match.player_of_match.value_counts()
man_of_the_match.columns = ['Batsman','man of match']
#man_of_the_match.sort_values('man of match', ascending=False, inplace=True)
mom= man_of_the_match.reset_index()


# Arrange batsman as per total run in IPL
runs= byb.groupby(['batsman'])['batsman_runs'].sum().reset_index()
runs.columns =['Batsman','Total run']
runs.sort_values('Total run', ascending=False, inplace=True)
runs.reset_index(inplace=True)
runs.drop('index',axis=1, inplace=True)
print(runs.head(10))
print()
print("Man of the Match")
print()
print(mom.head(10))


'''f, ax = plt.subplot(2,1,figsize=(5,5))

sns.barplot(data=runs.head(10), y='Batsman', x='Total run', ax=ax[0])
sns.barplot(data=mom.head(10), y='index' , x='player_of_match', ax=ax[1])
'''

In [ ]:
# Store top 10 players as per total run they score in list
players_name = runs['Batsman'].head(10).to_list()
# players_name

In [ ]:
# Dismissal of top 10 batsman

f, ax = plt.subplots(5,2,figsize=(25,35))
'''f.set_figheight(15)
f.set_figwidth(15)'''
ax = ax.flatten()

j=0
for i in players_name:
    player = (byb['batsman']==i)
    current_player = byb[player]
    current_player['dismissal_kind'].value_counts().plot.pie( ax=ax[j],autopct='%1.1f%%', shadow=True,rotatelabels=False)
    ax[j].set_title('Dismissal kind of '+i , fontweight= 'bold', fontsize=15)
    ax[j].set_ylabel('')
    ax[j].set_xlabel('')
    j=j+1 
   


#### Calculate dismissal of MS Dhoni
a= 'MS Dhoni'
player = (byb['batsman']=='MS Dhoni')
df_dhoni=byb[player]
df_dhoni['dismissal_kind'].value_counts().plot.pie(autopct='%1.1f%%', shadow=True, rotatelabels=False)
plt.title('Dismissal kind of '+ a, fontweight= 'bold', fontsize=15)
plt.show()

In [ ]:
# count single, double, triple, boundry and sixes for each player from top 10

for i in players_name:
    player = (byb['batsman']==i)
    current_player = byb[player]
    def count (current_player,runs):
        return len(current_player[current_player['batsman_runs']==runs])*runs
    print('Score distribution of '+ i)
    print('Runs scored from 1s :', count(current_player,1), '     And no of 1s = ',count(current_player,1)/1)
    print('Runs scored from 2s :', count(current_player,2), '      And no of 2s = ',count(current_player,2)/2)
    print('Runs scored from 3s :', count(current_player,3), '       And no of 3s = ',count(current_player,3)/3)
    print('Runs scored from 4s :', count(current_player,4), '     And no of 4s = ',count(current_player,4)/4)
    print('Runs scored from 6s :', count(current_player,6), '     And no of 6s = ',count(current_player,6)/6)
    print()


# Race Bar Chart Matplotlib

How to do this
**Rough Idea** 
> might have to use feature engineering then cumulative no and then 
1. cum_race_df= race_df.cumsum(axis=0)
2. Install : !pip install bar-chart-race
3. import bar_chart_race as bcr
            bcr.bar_chart_race(df= cum_race_df,figsize=(3, 3), title="CryptoCurrency Race")

In [ ]:
# Race chart graph for best batsman
rcg=byb[['batsman','batsman_runs']]
rcg

In [ ]:
# Race Bar Chart for Best team to win most IPL Match
match_race= match[['date','winner']]
match_race

# 3. Data Preparation for Prediction
* Label encoding 
* One-Hot encoding
Basically ML understands no better than string so we will convert team name to no.

# 4. Modeling
* splitting the dataset
* Choosing the ML algorithm
* Cross Validation
* Checking the model on test data

[Continue from here](https://www.youtube.com/watch?v=7rCW8fLdJGc)

# Glossory
1. SubPlot in Matplotlib
> Matplotlib’spyplot API has a convenience function called subplots() which acts as a utility wrapper and helps in creating common layouts of subplots, including the enclosing figure object, in a single call.
> Plt.subplots(nrows, ncols)
> The two integer arguments to this function specify the number of rows and columns of the subplot grid. The function returns a figure object and a tuple containing axes objects equal to nrows*ncols. Each axes object is accessible by its index. Here we create a subplot of 2 rows by 2 columns and display 4 different plots in each subplot.

